<a href="https://colab.research.google.com/github/rik53/kaggle_education/blob/main/kaggle_education.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import dask.dataframe as dd
import pandas as pd

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [85]:
train = dd.read_csv('/content/drive/MyDrive/data/train-001.csv')

In [86]:
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


Отфильтруем и оставим только те события, где были заданы вопросы и были даны верные ответы

In [4]:
df_question = train.loc[(train['content_type_id']	== 0)]

In [5]:
df_cor = df_question[df_question['answered_correctly'] == 1]

In [6]:
df_cor.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


Создадим pandas DF, где сгруппируем user_id и подсчитаем колличество верных ответов у пользователей

In [7]:
user_content_id_cor = df_cor.groupby('user_id').content_id.count().compute()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [8]:
user_content_id_cor.head()

,content_id
user_id,
115,32
124,7
2746,11
5382,84
8623,70


Отсортируем студентов по верно отвеченным вопросам в порядке возрастания

In [9]:
user_content_id_cor.sort_values()

,content_id
user_id,
868428301,1
447726620,1
122687953,1
1431665260,1
791976641,1
...,...
1047202059,13034
1842816145,13086
338684437,13416


Мы видем что наибошльшее количество верных ответов 14300, а наименьшее 1

In [10]:
print(f'Среднее количество верных ответов студентов {user_content_id_cor.mean()}')
print(f'Медианное значение верных ответов студентов {user_content_id_cor.median()}')

Среднее количество верных ответов студентов 165.81855541718556
Медианное значение верных ответов студентов 23.0


Мы видим большую разницу между средним и медианным значением, что говорит о наличии выбросов.

Отфильтруем и оставим только те события, где были заданы вопросы и были даны неверные ответы

In [11]:
df_err = df_question [df_question ['answered_correctly'] == 0]

In [12]:
df_err.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
10,10,255381,115,152,0,10,2,0,17000.0,False
11,11,280033,115,104,0,11,1,0,22000.0,False
12,12,302994,115,108,0,12,1,0,23000.0,False
17,17,418008,115,183,0,17,0,0,20000.0,False
20,20,490100,115,4,0,20,1,0,29000.0,False


Создадим pandas DF, где сгруппируем user_id и подсчитаем колличество неверных ответов у пользователей

In [13]:
user_content_id_err = df_err.groupby('user_id').content_id.count().compute()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [14]:
user_content_id_err.head()

,content_id
user_id,
115,14
124,23
2746,8
5382,41
8623,39


Отсортируем студентов по неверно отвеченным вопросам в порядке возрастания

In [15]:
user_content_id_err.sort_values()

,content_id
user_id,
43283008,1
2070051689,1
1313111726,1
483777693,1
1038082817,1
...,...
314180295,7588
7171715,7974
1509564249,8017


Мы видем что наибошльшее количество ошибочных ответов 8112, а наименьшее 1

In [16]:
print(f'Среднее количество верных ответов студентов {user_content_id_err.mean()}')
print(f'Медианное значение верных ответов студентов {user_content_id_err.median()}')

Среднее количество верных ответов студентов 86.48393803454104
Медианное значение верных ответов студентов 22.0


Мы видим большую разницу между средним и медианным значением, что говорит о наличии выбросов.

Создадим новый DataFrame в котором будут данные по правильным и неправильным ответам и подсчитаем долю правильных среди неправильных

In [17]:
df_agg = pd.DataFrame()
df_agg['correct_answer'] = user_content_id_cor
df_agg = df_agg.merge(user_content_id_err, on='user_id', how='inner')
df_agg = df_agg.rename(columns={'content_id':'wrong_answer'})
df_agg['correct_share'] = df_agg['correct_answer'] / (df_agg['correct_answer'] + df_agg['content_id'])

In [21]:
df_agg.head()

,correct_answer,content_id,correct_share
user_id,,,
115,32,14,0.695652
124,7,23,0.233333
2746,11,8,0.578947
5382,84,41,0.672000
8623,70,39,0.642202


Отсоритруем студентов по долям верных ответов

In [22]:
df_agg.correct_share.sort_values()

,correct_share
user_id,
793441695,0.031250
2136431267,0.032258
1413467893,0.032258
1571111838,0.032258
1986771392,0.033333
...,...
1914789481,0.978723
537921513,0.979592
631928207,0.983333


Наибольшая доля правильных ответов составляет 0.989011, наименьшая 0.031250

In [23]:
print(f'Среднее количество верных ответов студентов в долях {df_agg.correct_share.mean()}')
print(f'Медианное значение верных ответов студентов в долях {df_agg.correct_share.median()}')

Среднее количество верных ответов студентов в долях 0.5451970979528887
Медианное значение верных ответов студентов в долях 0.5714285714285714


Среднее и медианное значение не сильно различаются

In [24]:
df_lection = train.loc[(train['content_type_id']	== 1)]

In [25]:
df_lection.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
89,89,653762,2746,6808,1,14,-1,-1,NaN,False
117,117,10183847,5382,16736,1,21,-1,-1,NaN,False
212,212,1424348597,5382,30207,1,104,-1,-1,NaN,False
216,216,1425557777,5382,18545,1,121,-1,-1,NaN,False
295,295,405813029,8623,10540,1,59,-1,-1,NaN,False


Создадим pandas DF, где сгруппируем user_id и подсчитаем колличество просмотренных лекций студентами

In [26]:
user_lection = pd.DataFrame(df_lection.groupby('user_id').content_id.count().compute())

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [27]:
user_lection = user_lection.rename(columns={'content_id':'lections'})

In [28]:
user_lection.head()

,lections
user_id,
2746,1
5382,3
8623,3
12741,6
13134,7


Отсоритруем студентов по просмотренным лекциям

In [29]:
user_lection.sort_values(by='lections')

,lections
user_id,
2746,1
1013476128,1
1013483605,1
1013563997,1
1013621411,1
...,...
1066383521,386
2066672014,386
105239469,389


Наибольшее значение просмотренных лекций 397, наименьшее 1

In [30]:
print(f'Среднее количество просмотренных лекций студентов {user_lection.mean()}')
print(f'Медианное значение просмотренных лекций студентов {user_lection.median()}')

Среднее количество просмотренных лекций студентов lections    13.094609
dtype: float64
Медианное значение просмотренных лекций студентов lections    4.0
dtype: float64


Разница между средним и медианным значением достаточно велика, что может говорить о наличии выбросов

In [31]:
df_agg = df_agg.merge(user_lection, on='user_id', how='inner')

In [32]:
df_agg.head()

,correct_answer,content_id,correct_share,lections
user_id,,,,
2746,11,8,0.578947,1
5382,84,41,0.672000,3
8623,70,39,0.642202,3
12741,152,113,0.573585,6
13134,878,365,0.706356,7


Создадим pandas DF, где сгруппируем user_id и подсчитаем среднее значение времени в миллисекундах, которое потребовалось пользователю для ответа на каждый вопрос

In [33]:
user_question_elapsed_time = train.groupby('user_id').prior_question_elapsed_time.mean().compute()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


Переведем время в минуты

In [41]:
user_question_elapsed_time = user_question_elapsed_time * 1.6667e-5

In [42]:
user_question_elapsed_time.head()

,prior_question_elapsed_time
user_id,
115,0.332228
124,0.313223
2746,0.300932
5382,0.600818
8623,0.435132


Отсоритруем студентов по времени, которое потребовалось пользователю для ответа на каждый вопрос

In [43]:
user_question_elapsed_time.fillna(0).sort_values()

,prior_question_elapsed_time
user_id,
346289863,0.000000
118250768,0.000000
694377393,0.000000
1071441751,0.000000
1977213781,0.000000
...,...
1498819357,2.749297
790143287,2.991727
1762860853,3.738408


Наименьшее значение затраченного времени на ответ составляет 0 сек, наибольшее 3.85 мин.

In [44]:
print(f'Среднее количество просмотренных лекций студентов {user_question_elapsed_time.mean()}')
print(f'Медианное значение просмотренных лекций студентов {user_question_elapsed_time.median()}')

Среднее количество просмотренных лекций студентов 0.3982358725145506
Медианное значение просмотренных лекций студентов 0.37871841670769235


Среднее и медианное значение отличаются несильно, что говорит об отсутствии выбросов

In [45]:
df_agg = df_agg.merge(user_question_elapsed_time, on='user_id', how='inner')

In [46]:
df_agg.head()

,correct_answer,content_id,correct_share,lections,prior_question_elapsed_time
user_id,,,,,
2746,11,8,0.578947,1,0.300932
5382,84,41,0.672000,3,0.600818
8623,70,39,0.642202,3,0.435132
12741,152,113,0.573585,6,0.663928
13134,878,365,0.706356,7,0.360365



Создадим pandas DF, где сгруппируем user_id и подсчитаем колличество просмотренных пользователем объяснений и правильных ответов.

In [98]:
train_true = train[train['prior_question_had_explanation'] == 'True']

In [103]:
user_explanation = train_true.groupby('user_id').prior_question_had_explanation.count().compute()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [104]:
user_explanation.head()

,prior_question_had_explanation
user_id,
115,6
2746,11
5382,113
8623,96
8701,9


Отсоритруем студентов колличеству просмотренных пользователем объяснений и правильных ответов.

In [105]:
user_explanation.sort_values()

,prior_question_had_explanation
user_id,
1082500527,1
1227348544,1
1542673893,1
679250168,1
1961071578,1
...,...
2146130037,16139
1047202059,16153
1660941992,16376


Наименьшее количество просмотренных пользователем объяснений и правильных ответов 1, наибольшее 17526

In [107]:
print(f'Среднее количество просмотренных лекций студентов {user_explanation.mean()}')
print(f'Медианное значение просмотренных лекций студентов {user_explanation.median()}')

Среднее количество просмотренных лекций студентов 282.91171544025565
Медианное значение просмотренных лекций студентов 41.0


Среднее и медианное значение сильно отличаются, возможно есть выбросы

In [108]:
df_agg = df_agg.merge(user_explanation, on='user_id', how='inner')

In [109]:
df_agg.head()

,correct_answer,content_id,correct_share,lections,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,
2746,11,8,0.578947,1,0.300932,11
5382,84,41,0.672000,3,0.600818,113
8623,70,39,0.642202,3,0.435132,96
12741,152,113,0.573585,6,0.663928,250
13134,878,365,0.706356,7,0.360365,1233


In [111]:
questions = pd.read_csv('/content/drive/MyDrive/data/questions.csv')

In [119]:
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


Посмотрим распределение вопросов по секциям TOEIC теста

In [120]:
questions.part.value_counts()

,count
part,
5,5511
2,1647
3,1562
4,1439
6,1212
7,1160
1,992


Больше всего вопросов в 5 секции экзамена 5511, меньше всего в первой 992

In [121]:
lectures = pd.read_csv('/content/drive/MyDrive/data/lectures.csv')

In [127]:
lectures.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [123]:
lectures.describe()

,lecture_id,tag,part
count,418.000000,418.000000,418.000000
mean,16983.401914,94.480861,4.267943
std,9426.164660,53.586487,1.872424
min,89.000000,0.000000,1.000000
25%,9026.250000,50.250000,2.000000
50%,17161.500000,94.500000,5.000000
75%,24906.250000,140.000000,6.000000
max,32736.000000,187.000000,7.000000


In [124]:
lectures.describe(include='object')

,type_of
count,418
unique,4
top,concept
freq,222


Посмотрим распределение лекций по категориям

In [125]:
lectures.part.value_counts()

,count
part,
5,143
6,83
2,56
1,54
7,32
4,31
3,19


Меньше всего лекций в 3 категории 19, больше всего в 5 категории 143

Посмотрим распределение лекций по целям

In [126]:
lectures.type_of.value_counts()

,count
type_of,
concept,222
solving question,186
intention,7
starter,3


Меньше всего лекций в starter 3, больше всего в concept 222